In [317]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
from scipy import stats

df = pd.read_csv("cryptos.csv", index_col=0, parse_dates=True)
df.sort_index(inplace=True)
tickers = ["BTCUSDT", "ETHUSDT", "SOLUSDT", 'BNBUSDT']
n_long = 1
stop_loss = 0.05
twenty_four_hours = 24

df_all = df.copy()

In [298]:
start = twenty_four_hours*30
end = start + twenty_four_hours*30
df = df_all.iloc[start:end]

In [314]:
def max_drawdown(cum_returns):
    running_max = cum_returns.cummax()
    drawdown = (cum_returns - running_max) / running_max
    return drawdown.min()

In [318]:
def run_backtest(lookback, n_top, resample_hours):
    returns = df[tickers].pct_change()
    volatility = returns.rolling(window=lookback).std()
    momentum = df[tickers].pct_change(periods=lookback) / (volatility + 1e-8)
    
    momentum_clean = momentum.dropna()
    if len(momentum_clean) == 0:
        return 0.0, 0.0
    valid_start = momentum_clean.index[0]
    
    resample_str = f"{resample_hours}H"
    resampled_momentum = momentum.resample(resample_str).mean()
    
    signals = pd.DataFrame(0, index=resampled_momentum.index, columns=resampled_momentum.columns)
    positive_momentum = resampled_momentum > 0
    
    for idx in resampled_momentum.index:
        row = resampled_momentum.loc[idx]
        if not row.isna().all():
            positive_row = row.where(positive_momentum.loc[idx], np.nan)
            if not positive_row.isna().all():
                top_assets = positive_row.nlargest(n_top).index
                signals.loc[idx, top_assets] = 1
    
    signals = signals.shift(1).reindex(df.index).ffill()
    signals = signals.loc[signals.index >= valid_start]
    
    returns = df[tickers].pct_change().loc[lambda x: x.index >= valid_start]
    
    active_positions = signals.sum(axis=1)
    portfolio_returns = (returns * signals).sum(axis=1) / active_positions.replace(0, np.nan)
    portfolio_returns = portfolio_returns.fillna(0)
    
    equity_values = np.ones(len(portfolio_returns))
    peak_values = np.ones(len(portfolio_returns))
    active_flags = np.ones(len(portfolio_returns), dtype=bool)
    
    for i in range(1, len(portfolio_returns)):
        if active_flags[i-1]:
            equity_values[i] = equity_values[i-1] * (1 + portfolio_returns.iloc[i])
            peak_values[i] = max(peak_values[i-1], equity_values[i])
            if (equity_values[i] / peak_values[i] - 1) < -stop_loss:
                active_flags[i] = False
            else:
                active_flags[i] = True
        else:
            if portfolio_returns.iloc[i] != 0:
                active_flags[i] = True
                equity_values[i] = equity_values[i-1] * (1 + portfolio_returns.iloc[i])
                peak_values[i] = equity_values[i]
            else:
                active_flags[i] = False
                equity_values[i] = equity_values[i-1]
                peak_values[i] = peak_values[i-1]
    
    cumulative_returns = pd.Series(equity_values, index=portfolio_returns.index)
    final_return = cumulative_returns.iloc[-1] - 1
    
    running_maximum = cumulative_returns.expanding().max()
    drawdown = ((cumulative_returns - running_maximum) / running_maximum).min()
    
    return final_return, drawdown

# Unit test

In [311]:
lookback = 2
n_top = 1
resample_hours = 14
final_return, drawdown = run_backtest(lookback, n_top, resample_hours)

print(f"Lookback: {lookback}, n_top: {n_top}, Resample: {resample_hours}H | "
        f"Return: {final_return*100:.2f}%, Drawdown: {drawdown*100:.2f}%")

Lookback: 2, n_top: 1, Resample: 14H | Return: -14.49%, Drawdown: -47.07%


# Test Loop

In [88]:
results = []
for lookback in range(1, 2):
    for n_top in range(1, 2):
        for resample_hours in range(13, 16):
            final_return, drawdown = run_backtest(lookback, n_top, resample_hours)
            results.append({
                'lookback': lookback,
                'n_top': n_top,
                'resample_hours': resample_hours,
                'final_return': final_return,
                'max_drawdown': drawdown
            })
            # print(f"Lookback: {lookback}, n_top: {n_top}, Resample: {resample_hours}H | "
            #       f"Return: {final_return*100:.2f}%, Drawdown: {drawdown*100:.2f}%")
results_df = pd.DataFrame(results).sort_values(by="final_return", ascending=False)
results_df

,lookback,n_top,resample_hours,final_return,max_drawdown
1,1,1,14,0.409520,-0.121652
0,1,1,13,0.158549,-0.157377
2,1,1,15,0.134531,-0.155342


In [87]:
results_df[results_df["lookback"]==8]

,lookback,n_top,resample_hours,final_return,max_drawdown
83,8,1,12,0.361782,-0.064043
81,8,1,10,0.347556,-0.049677
77,8,1,6,0.232704,-0.053963
78,8,1,7,0.230605,-0.086410
75,8,1,4,0.224272,-0.096298
76,8,1,5,0.185026,-0.078194
82,8,1,11,0.177482,-0.053622
80,8,1,9,0.174050,-0.080497
79,8,1,8,0.167127,-0.080497
74,8,1,3,0.095908,-0.112518


In [ ]:
def run_grid_search():
    results = []
    for lookback in range(1, 13):
        for n_top in range(1, 2):
            for resample_hours in range(1, 25):
                final_return, drawdown = run_backtest(lookback, n_top, resample_hours)
                results.append({
                    'lookback': lookback,
                    'n_top': n_top,
                    'resample_hours': resample_hours,
                    'final_return': final_return,
                    'max_drawdown': drawdown
                })
                #print(f"Lookback: {lookback}, n_top: {n_top}, Resample: {resample_hours}H | "
                #      f"Return: {final_return*100:.2f}%, Drawdown: {drawdown*100:.2f}%")
    results_df = pd.DataFrame(results).sort_values(by="final_return", ascending=False).reset_index(drop=True)
    return results_df

# Test weekly

In [325]:
train_period_days, test_period_days, start_timestamp, total, previous_iteration_return, final_return = 25, 15, 0, 1, 0, 0
print("train: ", train_period_days, "\ttest: ", test_period_days)

for i in range(0, 50):
    train_start = twenty_four_hours * start_timestamp
    train_end = train_start + twenty_four_hours * train_period_days
    test_start = train_end
    test_end = test_start + twenty_four_hours * test_period_days

    df_train = df_all.iloc[train_start:train_end]
    df_test = df_all.iloc[test_start:test_end]

    df = df_train
    results_df = run_grid_search()

    lookback = results_df.loc[0, 'lookback']
    n_top = results_df.loc[0, 'n_top']
    resample_hours = results_df.loc[0, 'resample_hours']

    df = df_test
    final_return, drawdown = run_backtest(lookback, n_top, resample_hours)

    if previous_iteration_return >= 0:
        total *= (1 + final_return)
    else:
        total *= (1 + final_return)

    print(f"Lookback: {lookback}, n_top: {n_top}, Resample: {resample_hours}H | "
          f"Return: {final_return*100:.2f}%, Drawdown: {drawdown*100:.2f}%, Total: {total*100:.2f}%")

    previous_iteration_return = final_return
    start_timestamp += test_period_days

train:  25 	test:  15
Lookback: 10, n_top: 1, Resample: 11H | Return: 9.46%, Drawdown: -7.31%, Total: 109.46%
Lookback: 8, n_top: 1, Resample: 8H | Return: 5.02%, Drawdown: -6.31%, Total: 114.96%
Lookback: 6, n_top: 1, Resample: 22H | Return: 46.16%, Drawdown: -10.62%, Total: 168.03%
Lookback: 8, n_top: 1, Resample: 23H | Return: 21.28%, Drawdown: -15.75%, Total: 203.79%
Lookback: 3, n_top: 1, Resample: 9H | Return: -14.01%, Drawdown: -16.39%, Total: 175.23%
Lookback: 5, n_top: 1, Resample: 6H | Return: 9.31%, Drawdown: -10.35%, Total: 191.54%
Lookback: 5, n_top: 1, Resample: 6H | Return: -3.48%, Drawdown: -14.94%, Total: 184.88%
Lookback: 4, n_top: 1, Resample: 2H | Return: 10.31%, Drawdown: -9.17%, Total: 203.94%


KeyboardInterrupt: 

In [32]:
train_period_days = 30
test_period_days = 30
start_timestamp, total, previous_iteration_return, final_return = 0, 1, 0, 0

for i in range(0, 10):
    train_start = twenty_four_hours * start_timestamp
    train_end = train_start + twenty_four_hours * train_period_days
    test_start = train_end
    test_end = test_start + twenty_four_hours * test_period_days

    print(train_start/24)
    print(train_end/24)
    print(test_start/24)
    print(test_end/24)
    print('________________________')

    start_timestamp += test_period_days


0.0
30.0
30.0
60.0
________________________
30.0
60.0
60.0
90.0
________________________
60.0
90.0
90.0
120.0
________________________
90.0
120.0
120.0
150.0
________________________
120.0
150.0
150.0
180.0
________________________
150.0
180.0
180.0
210.0
________________________
180.0
210.0
210.0
240.0
________________________
210.0
240.0
240.0
270.0
________________________
240.0
270.0
270.0
300.0
________________________
270.0
300.0
300.0
330.0
________________________
